In [1]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
train_g = pd.read_csv('train_groups.csv')
test_g  = pd.read_csv('test_groups.csv')
t = pd.concat([test_g, train_g], axis=0, sort=False)# объединение для получения всех айди
ng = pd.read_csv('ng.csv')
test = pd.merge(t, ng, on='doc_id', how='left')# сопоставление тайтд -айди
test = test.replace(np.nan, 'noinformation')
print(test.shape)
test.head()

(28317, 5)


,pair_id,group_id,doc_id,target,title
0,11691,130,6710,noinformation,прописать админк кс 16 друг youtube
1,11692,130,4030,noinformation,скачать sglrp доработка слив мода mysql rp ...
2,11693,130,5561,noinformation,прописать админк кс 16 counterstrike каталог...
3,11694,130,4055,noinformation,прописать простой админк кс 1 6
4,11695,130,4247,noinformation,подбор админовый сервер код4 архив форум ozone


In [3]:
h_vectorizer = HashingVectorizer(n_features=2**11, dtype = 'float32' ,ngram_range=(1, 3))
X = h_vectorizer.fit_transform(test.title) # постороение хэшей на файле без пропусков
h_vec = pd.DataFrame(X.toarray())
h_vec.shape

(28317, 2048)

In [29]:
a = pd.concat([test, h_vec], axis = 1)

In [30]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=1)

In [55]:
res = []

In [56]:

for i in range(1,130):
    g = a[a.group_id == i]
    g = g[g.target == 1]
    label = km.fit(g.drop(['pair_id', 'group_id', 'doc_id', 'title', 'target'], 1)).cluster_centers_
    res.append(label[0])
    

In [57]:
res = np.asarray(res)
res.shape

(129, 2048)

In [68]:
import scipy.spatial
distances = scipy.spatial.distance.cdist(a.drop(['pair_id', 'group_id', 'doc_id', 'title', 'target'], 1), res,'cosine')

In [69]:
distances.shape

(28317, 129)

In [71]:
Dist = pd.concat([test, pd.DataFrame(distances)], axis = 1)

In [ ]:
Dist.to_csv('dist')